# Real-world Data Wrangling

In [41]:
!python -m pip install kaggle==1.6.12

In [42]:
## !pip install --target=/workspace ucimlrepo numpy==1.24.3

**Note:** Restart the kernel to use updated package(s).

In [44]:
import pandas as pd
import numpy as np
import requests

## 1. Gather data

### **1.1.** Problem Statement
In this project, we will investigate the educational attainment levels among individuals employed in computer-related occupations in King County, Washington, by gathering, cleaning, and analyzing relevant data to identify trends, distributions, and potential correlations within the local workforce

### **1.2.** Gather at least two datasets using two different data gathering methods

#### **Dataset 1**

Type: XLS file

Method: The data was gathered using the "Downloading files" method from U.S Bureau of Labor Statistics (Occupational Employment and Wage Statistics tables)

Dataset variables:

*   *AREA:* Area code
*   *AREA_TITLE:* Title of the metropolitan area
*   *OCC_CODE:* Occupational Code
*   *OCC_TITLE:* Occupational Title
*   *A_MEAN:* Mean Annual Wage

In [49]:
## Load OEW Dataset to access king county/seattle-tacoma area dataset
OEW_data = pd.read_excel('MSA_M2023_dl.xlsx')

In [50]:
## Check the datasets' first 5 rows
OEW_data.head(5)

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,18.6,28.03,40.08,22750,28710,38680,58310,83360,NaN,NaN
1,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,39.09,58.27,81.79,42080,59290,81310,121200,170130,NaN,NaN
2,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,97.28,#,#,98490,135280,202340,#,#,NaN,NaN
3,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,37.19,57.85,86.97,34440,49430,77360,120330,180900,NaN,NaN
4,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2021,Marketing Managers,...,45.31,72.66,97.35,60130,74980,94240,151130,202490,NaN,NaN


#### Dataset 2

Type: API

Method: The data was gathered using the "API" method from the United States Census Bureau Public Use Microdata Sample site

Dataset variables:

*   *SOCP:* Standard Occupational Classification (SOC) codes for 2018 and later based oin 2018 SOC codes
*   *SCHL:* Educational Attainment
*   *SCHL_RC1:* Educational Attainment recode 

In [217]:
## Access PUMS API for census information for king county/seattle-tacoma dataset
url = 'https://api.census.gov/data/2023/acs/acs1/pums?get=PWGTP,SOCP,SCHL_RC1,SCHL&ucgid=795P200US5323304&recode+SCHL_RC1=%7B%22b%22:%22SCHL%22,%22d%22:%5B%5B%220%22,%2201%22,%2202%22,%2203%22,%2204%22,%2205%22,%2206%22,%2207%22,%2208%22,%2209%22,%2210%22,%2211%22,%2212%22,%2213%22,%2214%22,%2215%22%5D,%5B%2216%22,%2217%22%5D,%5B%2218%22,%2219%22%5D,%5B%2220%22,%2221%22%5D,%5B%2222%22,%2223%22,%2224%22%5D%5D%7D'
PUMS_response = requests.get(url)
PUMS_response.raise_for_status()

## Get the json
PUMS_response_data = PUMS_response.json()

## Create dataframe from json
PUMS_data = pd.DataFrame(PUMS_response_data)

In [219]:
## Check the data returns
print(PUMS_data.head(5))

       0       1         2     3      4      5
0  PWGTP    SOCP  SCHL_RC1  SCHL   PUMA  STATE
1     47       N         3    19  23304     53
2     74  537062         3    19  23304     53
3     54       N         4    21  23304     53
4     18       N         4    21  23304     53


In [97]:
# Create subset of dataset needed for analysis
OEW_subset = OEW_data[['AREA', 'OCC_CODE', 'OCC_TITLE', 'A_MEAN']].reindex()

In [101]:
# Inspecting the dataframe visually
OEW_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149591 entries, 0 to 149590
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   AREA       149591 non-null  int64 
 1   OCC_CODE   149591 non-null  object
 2   OCC_TITLE  149591 non-null  object
 3   A_MEAN     149591 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.6+ MB


#### MESSY DATA aka Structural Issues/method equal visual assessment
#### A_MEAN should be float/int64
#### OCC_CODE should be string - check that it can be converted to int64/float
#### OCC_TITLE should be string

In [145]:
seattle_area = 42660
OEW_seattle_data = OEW_subset[OEW_subset['AREA'] == seattle_area]

In [147]:
OEW_seattle_data.sample(20)

,AREA,OCC_CODE,OCC_TITLE,A_MEAN
118936,42660,13-1041,Compliance Officers,93250
119591,42660,53-2022,Airfield Operations Specialists,76780
119266,42660,33-2011,Firefighters,92560
119308,42660,39-1014,First-Line Supervisors of Entertainment and Re...,65040
119150,42660,27-1025,Interior Designers,80950
119440,42660,47-3011,"Helpers--Brickmasons, Blockmasons, Stonemasons...",73930
119363,42660,43-3071,Tellers,51000
119409,42660,45-2093,"Farmworkers, Farm, Ranch, and Aquacultural Ani...",40260
118893,42660,00-0000,All Occupations,86520
118894,42660,11-0000,Management Occupations,174450


In [137]:
comsci_startwith = '15-'
OEW_seattle_comsci = OEW_seattle_data[ OEW_seattle_data['OCC_CODE'].str.startswith(comsci_startwith)]

In [151]:
OEW_seattle_comsci.sample(10)

,AREA,OCC_CODE,OCC_TITLE,A_MEAN
118977,42660,15-1299,"Computer Occupations, All Other",137130
118972,42660,15-1251,Computer Programmers,167190
118968,42660,15-1241,Computer Network Architects,167420
118980,42660,15-2031,Operations Research Analysts,106620
118963,42660,15-1211,Computer Systems Analysts,138410
118976,42660,15-1255,Web and Digital Interface Designers,128650
118965,42660,15-1221,Computer and Information Research Scientists,211410
118970,42660,15-1243,Database Architects,159020
118979,42660,15-2021,Mathematicians,153070
118962,42660,15-0000,Computer and Mathematical Occupations,144530


In [164]:
OEW_seattle_comsci.duplicated()

118962    False
118963    False
118964    False
118965    False
118966    False
118967    False
118968    False
118969    False
118970    False
118971    False
118972    False
118973    False
118974    False
118975    False
118976    False
118977    False
118978    False
118979    False
118980    False
118981    False
118982    False
118983    False
dtype: bool

In [141]:
# Create subset of dataset needed for analysis
PUMS_subset = PUMS_data[['SOCP','SCHL_RC1', 'SCHL']].reindex()

In [143]:
PUMS_subset.sample(20)

,SOCP,SCHL_RC1,SCHL
1192,N,4,21
281,N,4,20
558,N,3,19
1224,472111,3,18
367,1720XX,5,22
1041,113031,5,22
691,N,1,2
1462,111021,4,21
1099,N,2,16
185,311122,4,21


In [170]:
PUMS_subset.duplicated().value_counts()

True     1292
False     482
Name: count, dtype: int64

### TODO ###
### - DIRTY DATA - Remove SOCP = N 'reason invalid entry in this context (visal assessment) ###
### - Turn SCHL_RC1 into actuall 
#### SCHL_RC1 == 0 => [No high school diploma] => SCHL == [0-15]
#### SCHL_RC1 == 1 => [high school diploma] => SCHL == [16-17]
#### SCHL_RC1 == 2 => [Completed Some College] => SCHL == [18-19]
#### SCHL_RC1 == 3 => [Graduated College] => SCHL == [20-21]
#### SCHL_RC1 == 4 => [Completed Advance Degree] => SCHL == [22-24]
#### Data Quality issue - completeness address by above?

### Dimensions of data quality ###

**Completeness** is a metric that helps you understand whether your data is sufficient to answer interesting questions or solve your problem.
**Validity** is a metric helping you understand how well your data conforms to a defined set of rules for data, also known as a schema.
**Accuracy** is a metric that helps you understand whether your data accurately represents the reality it aims to depict.
**Consistency** is a metric that helps you understand two things: whether your data follows a standard format and whether your data’s info matches with information from other data sources.
**Uniqueness** is a metric that helps you understand whether there are duplicate or overlapping values in your data.

## validity - check that salary is within range and there are no outliers ## std +-1 ) (programmatic) ##
## DATA QUALITY check for uniqueness (programmatic assessment) ##

### requirements for tidiness: ###
Each **variable** forms a column
Each **observation** forms a row
Each type of **observational unit** forms a table

## 2. Assess data

Assess the data according to data quality and tidiness metrics using the report below.

List **two** data quality issues and **two** tidiness issues. Assess each data issue visually **and** programmatically, then briefly describe the issue you find.  **Make sure you include justifications for the methods you use for the assessment.**

### Quality Issue 1:

In [225]:
### header is the first row of data 
print(PUMS_data.head(5))

       0       1         2     3      4      5
0  PWGTP    SOCP  SCHL_RC1  SCHL   PUMA  STATE
1     47       N         3    19  23304     53
2     74  537062         3    19  23304     53
3     54       N         4    21  23304     53
4     18       N         4    21  23304     53


Issue and justification: *column header* is the first row of dataset

### Quality Issue 2:

In [232]:
# Inspecting the dataframe programmatically
PUMS_data.describe()

,0,1,2,3,4,5
count,1775,1775,1775,1775,1775,1775
unique,265,219,6,26,2,2
top,66,N,4,21,23304,53
freq,40,691,625,552,1774,1774


Issue and justification:
*SOCP has a value N which is invalid entry in this context and should be removed from the dataset*

### Tidiness Issue 1:

In [310]:
#Inspecting the dataframe visually
OEW_data['A_MEAN'].value_counts()

A_MEAN
*         1487
#          144
54330       50
46110       48
40820       47
          ... 
207260       1
238120       1
193040       1
161570       1
295110       1
Name: count, Length: 15760, dtype: int64

Issue and justification: *For my analysis, A_MEAN should be a float/int64 and not an object. It also has values '\*' and '#' which are invalid*


### Tidiness Issue 2: 

In [325]:
## Inspecting the dataframe visually
PUMS_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1775 non-null   object
 1   1       1775 non-null   object
 2   2       1775 non-null   object
 3   3       1775 non-null   object
 4   4       1775 non-null   object
 5   5       1775 non-null   object
dtypes: object(6)
memory usage: 83.3+ KB


In [327]:
#Inspecting the dataframe programmatically
PUMS_data.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

Issue and justification: *FILL IN*

## 3. Clean data
Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [248]:
# Create copies of the datasets to ensure the raw dataframes 
PUMS_data_copy = PUMS_data.copy()
OEW_data_copy = OEW_data.copy()

### **Quality Issue 1: FILL IN**

In [256]:
# Cleaning strategy - Fix issue with column headers
columns = PUMS_response_data[0]
rows = PUMS_response_data[1:]
PUMS_data_cleaned = pd.DataFrame(rows, columns=columns)

In [258]:
# Validate the cleaning was successful
PUMS_data_cleaned.head(5)

,PWGTP,SOCP,SCHL_RC1,SCHL,PUMA,STATE
0,47,N,3,19,23304,53
1,74,537062,3,19,23304,53
2,54,N,4,21,23304,53
3,18,N,4,21,23304,53
4,47,N,3,19,23304,53


Justification: *Fixing the column headers improves that handling when trying to combine the data from the two datasets to use column names instead of numbers*

### **Quality Issue 2: FILL IN**

In [270]:
# Cleaning strategy - remove all rows with SOCP value equals 'N'
PUMS_data_cleaned = PUMS_data_cleaned[PUMS_data_cleaned['SOCP'] != 'N']

In [273]:
# Validate the cleaning was successful
PUMS_data_cleaned.sample(20)

,PWGTP,SOCP,SCHL_RC1,SCHL,PUMA,STATE
1137,168,253041,4,21,23304,53
534,6,353023,3,19,23304,53
1215,374,151252,3,19,23304,53
1232,74,151252,4,21,23304,53
1419,75,412031,4,21,23304,53
385,128,151252,4,21,23304,53
522,121,252020,5,22,23304,53
354,112,131161,4,21,23304,53
1392,35,132011,5,22,23304,53
1177,56,252020,5,22,23304,53


Justification: *SOCP has a row with value 'N' which is invalid entry in this context and after the analysis if not removed*

### **Tidiness Issue 1: FILL IN**

In [321]:
#Cleaning strategy - Replace rows with '*' & '#' in A_MEAN to np.nan and column to float/int
pd.set_option('future.no_silent_downcasting', True)

OEW_data_copy['A_MEAN'] = OEW_data_copy['A_MEAN'].replace(['*','#'], [np.nan, np.nan])
OEW_data_copy['A_MEAN'].astype(float)

0          49780.0
1         100690.0
2         240020.0
3          98480.0
4         116290.0
            ...   
149586     44550.0
149587     45740.0
149588     35630.0
149589     38900.0
149590     52670.0
Name: A_MEAN, Length: 149591, dtype: float64

In [314]:
#FILL IN - Validate the cleaning was successful
OEW_data_copy['A_MEAN'].value_counts()

A_MEAN
54330.0     50
46110.0     48
40820.0     47
38380.0     47
47300.0     47
            ..
143790.0     1
158300.0     1
147040.0     1
118300.0     1
295110.0     1
Name: count, Length: 15758, dtype: int64

Justification: *Replacing this values with np.nan exlcudes them from calculations of mean and sum by default*

### **Tidiness Issue 2: FILL IN**

In [ ]:
#FILL IN - Apply the cleaning strategy

In [ ]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Remove unnecessary variables and combine datasets**

Depending on the datasets, you can also peform the combination before the cleaning steps.

In [ ]:
#FILL IN - Remove unnecessary variables and combine datasets

## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [ ]:
#FILL IN - saving data

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [ ]:
#Visual 1 - FILL IN

*Answer to research question:* FILL IN

In [ ]:
#Visual 2 - FILL IN

*Answer to research question:* FILL IN

### **5.2:** Reflection
In 2-4 sentences, if you had more time to complete the project, what actions would you take? For example, which data quality and structural issues would you look into further, and what research questions would you further explore?

*Answer:* FILL IN